In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings("ignore")

import nltk
from thefuzz import fuzz
from thefuzz import process

In [2]:
%%time
df=pd.read_csv('wiki_movie_plots_deduped.csv')
df['Genre'].nunique()

Wall time: 662 ms


2265

In [3]:
def genre(data):
    df1 = pd.DataFrame(columns = ['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast','Genre', 'Wiki Page', 'Plot', 'Genre_1','Genre_updated'])
    for i,row in data.iterrows():
        if len(row['Genre_1'].split(','))>1:
            for i in row['Genre_1'].split(','):
               # print(i)
                df=pd.DataFrame(row).T
                df['Genre_updated']=i
                df1=pd.concat([df1,df])
        else:
            df=pd.DataFrame(row).T
            df['Genre_updated']=df['Genre_1']
            df1=pd.concat([df1,df])
    df['Genre_updated']=df['Genre_updated'].apply(lambda x: x.strip())
    return df1.reset_index(drop=True)

In [4]:

df['Genre_1']= df['Genre'].str.replace("unknown", "")


In [5]:
dax=df['Genre_1'].value_counts().rename_axis('Genres_1').reset_index(name='counts')
dax[:40]

,Genres_1,counts
0,,6083
1,drama,5964
2,comedy,4379
3,horror,1167
4,action,1098
5,thriller,966
6,romance,923
7,western,865
8,crime,568
9,adventure,526


In [6]:
[i for i in dax['Genres_1'].to_list() if len(i.split())<2][1:18]

['drama',
 'comedy',
 'horror',
 'action',
 'thriller',
 'romance',
 'western',
 'crime',
 'adventure',
 'musical',
 'mystery',
 'war',
 'animation',
 'sci-fi',
 'family',
 'fantasy',
 'animated']

In [7]:
choices = [i for i in dax['Genres_1'].to_list() if len(i.split())<2][1:18]


In [8]:
li=[]
def list_prep(data):
    for i in data:
        if i[1]==[]:
            choices.append(i[0])
        else:
            li.append(i[0])
        

In [9]:
%%time
for i in range(10,len(dax['Genres_1'].to_list()),10):
    inputt=dax['Genres_1'].to_list()[:i]
    data=[[i,process.extractBests(i, choices,score_cutoff=80,limit=20)] for i in inputt if i not in choices if i not in li]
    list_prep(data)


Wall time: 2min


In [10]:
df['Genre_1']=df['Genre_1'].apply(lambda x: x.strip())

In [11]:
[[row['Genres_1'],row['counts']] for i,row in dax.iterrows() if row['Genres_1'] in choices ]

[['', 6083],
 ['drama', 5964],
 ['comedy', 4379],
 ['horror', 1167],
 ['action', 1098],
 ['thriller', 966],
 ['romance', 923],
 ['western', 865],
 ['crime', 568],
 ['adventure', 526],
 ['musical', 467],
 ['science fiction', 418],
 ['film noir', 345],
 ['mystery', 310],
 ['war', 273],
 ['animation', 264],
 ['sci-fi', 221],
 ['family', 217],
 ['fantasy', 204],
 ['animated', 195],
 ['biography', 136],
 ['anime', 112],
 ['suspense', 104],
 ['social', 82],
 ['historical', 77],
 ['documentary', 73],
 ['serial', 71],
 ['biopic', 57],
 ['superhero', 48],
 ['martial arts', 46],
 ['short', 30],
 ['tokusatsu', 28],
 [' ', 28],
 ['spy', 27],
 ['dramedy', 24],
 ['sports', 23],
 ['mythology', 22],
 ['biographical', 20],
 ['folklore', 17],
 ['masala', 16],
 ['rom com', 14],
 ['kaiju', 13],
 ['propaganda', 13],
 ['slapstick', 12],
 ['slasher', 12],
 ['devotional', 12],
 ['disaster film', 11],
 ['epic', 10],
 ['swashbuckler', 10],
 ['wuxia', 9],
 ['parody', 8],
 ['satire', 8],
 ['children', 7],
 ['dete

In [12]:
replace_dict={
   "science_friction":"sci_fi",
    "sci fi":"sci_fi",
    "science friction":"sci_fi",
    "science fiction":"sci_fi",
    "sci-fi":"sci_fi",
    "amine":"animated",
    "animation":"animated",
    "cartoon":"animated",
    "tokusatsu": "animated",
    "superhero": "animated",
    "suspense": "thriller",
    "dramedy": "drama",
    "blaxploitation": "exploitation",
    "biopic": "biography",
    "bio-pic": "biography",
    "biographical": "biography",
    "kung fu": "martial_arts",
    "martial arts": "martial_arts",
    "kung fu": "martial_arts",
    "kung-fu": "martial_arts",
    "film": "",
    "neo-noir": "noir",
    "devotional": "religious",
    "children": "teen",
    "porno": "adult",
    "history": "historical",
    "rom com": "romance-comedy",
    "rom-com": "romance-comedy",
    "love": "romance",
    "periodic":"historical",
    "period":"historical",
    "football":"sports",
    "youth":'teen',
    "sexual":"adult",
    "/":" ",
    '-':' ',
    "slice of life":'slice_of_life',
    "love":"romance",
}

In [13]:
%%time
for key,items in replace_dict.items():
    df['Genre_1']= df['Genre_1'].str.replace(key,items)
df['Genre_1']=df['Genre_1'].apply(lambda x: x.strip())

Wall time: 583 ms


In [14]:
%%time
df=genre(df)

Wall time: 2min 15s


In [15]:
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre_1,Genre_updated
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",,
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",,
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",,
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,,
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,,
...,...,...,...,...,...,...,...,...,...,...
38913,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ...",,
38914,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...",comedy,comedy
38915,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...",comedy,comedy
38916,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,romantic comedy,romantic comedy


In [16]:
df['Genre_updated']=df['Genre_updated'].apply(lambda x: x.strip())

In [17]:
dax=df['Genre_updated'].value_counts().rename_axis('Genres_updated').reset_index(name='counts')
choices = [i for i in dax['Genres_updated'].to_list() if len(i.split())<2][0:18]
li=[]
def list_prep(data):
    for i in data:
        if i[1]==[]:
            choices.append(i[0])
        else:
            li.append(i[0])

In [18]:
%%time
for i in range(10,len(dax['Genres_updated'].to_list()),10):
    inputt=dax['Genres_updated'].to_list()[:i]
    data=[[i,process.extractBests(i, choices,score_cutoff=80,limit=20)] for i in inputt if i not in choices if i not in li]
    list_prep(data)

Wall time: 1min 1s


In [19]:
choices_final=[row['Genres_updated'] for i,row in dax.iterrows() if row['Genres_updated'] in choices if row['counts']>30]

In [20]:
choices_final

['drama',
 '',
 'comedy',
 'action',
 'romance',
 'thriller',
 'horror',
 'western',
 'crime',
 'sci_fi',
 'adventure',
 'musical',
 'animated',
 'family',
 'mystery',
 'biography',
 'fantasy',
 'war',
 'noir',
 'historical',
 'social',
 'anime',
 'documentary',
 'spy',
 'serial',
 'sports',
 'martial_arts',
 'teen',
 'short']

In [21]:
[[row['Genres_updated'],row['counts']] for i,row in dax.iterrows() if row['Genres_updated'] in choices_final ]
[row['Genres_updated'] for i,row in dax.iterrows() if row['Genres_updated'] in choices if row['counts']>30]

['drama',
 '',
 'comedy',
 'action',
 'romance',
 'thriller',
 'horror',
 'western',
 'crime',
 'sci_fi',
 'adventure',
 'musical',
 'animated',
 'family',
 'mystery',
 'biography',
 'fantasy',
 'war',
 'noir',
 'historical',
 'social',
 'anime',
 'documentary',
 'spy',
 'serial',
 'sports',
 'martial_arts',
 'teen',
 'short']

In [22]:

def genre_list(x):
    
    if x in choices_final:
        return x
    elif process.extractBests(x, choices_final,score_cutoff=80,limit=20) !=[]:
        #print(choices_final)
        a=process.extractBests(x, choices_final,score_cutoff=80,limit=20)
        #print(a)
        return ','.join([a[i][0] for i in range(len(a))])
    else:
        return ''

In [23]:
genre_list('drama romance')

'drama,romance'

In [24]:
df['Genre_updated']=df['Genre_updated'].apply(lambda x: genre_list(x))

In [25]:
def genre_updated(data):
    df1 = pd.DataFrame(columns = ['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast','Genre', 'Wiki Page', 'Plot','Genre_1','Genre_updated','Genre_updated1'])
    for i,row in data.iterrows():
        if len(row['Genre_updated'].split(','))>1:
            for i in row['Genre_updated'].split(','):
               # print(i)
                df=pd.DataFrame(row).T
                df['Genre_updated1']=i
                df1=pd.concat([df1,df])
        else:
            df=pd.DataFrame(row).T
            df['Genre_updated1']=df['Genre_updated']
            df1=pd.concat([df1,df])
    df['Genre_updated1']=df['Genre_updated1'].apply(lambda x: x.strip())       
    return df1.reset_index(drop=True)

In [26]:
%%time
new_df=genre_updated(df)

Wall time: 2min 48s


In [27]:
new_df['Genre_updated1'].value_counts()

drama           9513
comedy          7295
                6950
action          2473
romance         1986
thriller        1924
crime           1604
horror          1593
western          974
musical          960
sci_fi           932
animated         905
adventure        868
family           635
war              604
fantasy          561
mystery          481
biography        457
noir             358
historical       285
short            242
social           223
anime            143
spy              142
martial_arts     133
documentary      130
sports           126
teen             100
serial            83
Name: Genre_updated1, dtype: int64

In [28]:
new_df.drop_duplicates().drop(['Genre_1','Genre_updated'],axis=1).rename(columns={'Genre_updated1': 'Genre_updated'}).to_csv('wiki_movie_plots_deduped_cleaned_genre.csv',index=False)

In [29]:
new_df[new_df['Genre']!=new_df['Genre_updated1']]

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre_1,Genre_updated,Genre_updated1
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",,,
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",,,
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",,,
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,,,
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,,,
...,...,...,...,...,...,...,...,...,...,...,...
42673,2014,Winter Sleep,Turkish,Director: Nuri Bilge Ceylan,Director: Nuri Bilge Ceylan\r\nCast: Haluk Bil...,unknown,https://en.wikipedia.org/wiki/Winter_Sleep_(film),"Aydın, a former actor, owns a mountaintop hote...",,,
42674,2014,Sivas,Turkish,Director: Kaan Müjdeci,Director: Kaan Müjdeci\r\nCast: Dogan Izci,unknown,https://en.wikipedia.org/wiki/Sivas_(film),The film follows an eleven-year-old boy named ...,,,
42675,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ...",,,
42678,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,romantic comedy,comedy,comedy


In [30]:
new_df[(new_df['Genre_updated1']=='') & (new_df['Genre']!=new_df['Genre_updated1'])]

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre_1,Genre_updated,Genre_updated1
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",,,
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",,,
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",,,
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,,,
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,,,
...,...,...,...,...,...,...,...,...,...,...,...
42627,2009,Orada,Turkish,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Orada,When the matriarch of an Istanbul family dies ...,,,
42672,2014,Mandıra Filozofu,Turkish,Director: Müfit Can Saçıntı,Director: Müfit Can Saçıntı\r\nCast: Rasim Özt...,unknown,https://en.wikipedia.org/wiki/Mand%C4%B1ra_Fil...,Cavit an ambitious industralist in İstanbul pl...,,,
42673,2014,Winter Sleep,Turkish,Director: Nuri Bilge Ceylan,Director: Nuri Bilge Ceylan\r\nCast: Haluk Bil...,unknown,https://en.wikipedia.org/wiki/Winter_Sleep_(film),"Aydın, a former actor, owns a mountaintop hote...",,,
42674,2014,Sivas,Turkish,Director: Kaan Müjdeci,Director: Kaan Müjdeci\r\nCast: Dogan Izci,unknown,https://en.wikipedia.org/wiki/Sivas_(film),The film follows an eleven-year-old boy named ...,,,


In [31]:
new_df[(new_df['Genre_updated1']=='') & (new_df['Genre']!=new_df['Genre_updated1'])]['Genre'].unique()

array(['unknown', 'epic', 'sexual hygiene/exploitation film',
       'propaganda', 'ww1 propaganda', 'swashbuckler', 'race film',
       'operetta', 'detective', 'gangster', 'biblical', 'race',
       'charlie chan', 'aviation', 'experimental', 'educational',
       'slapstick', 'espionage', 'horror, 3-d', 'thriller, 3-d',
       'sci-fi, 3-d', 'western, 3-d', 'science-fiction', 'p.o.w.',
       'romance, trilogy', 'paramount. biblical epic', 'disaster film',
       'drama, epic', 'disaster', 'adult', 'epic, biography', 'travel',
       'war, satire', 'comedy, satire', 'slasher', 'comedy, parody',
       'outlaw biker film', 'biker film', 'anthology', 'biker',
       'war, biker', 'animated, adult', 'disaster film, adventure',
       'adult film', 'science-fiction, thriller', 'adult, horror',
       'drama, television miniseries', 'historical, erotic',
       'disaster film, sci-fi', 'comedy, anthology', 'animation, adult',
       '16\xa0mm film', 'parody', 'horror, slasher', 'comedy, 

In [32]:
new_df[(new_df['Genre_updated1']=='') & (new_df['Genre_1']!=new_df['Genre_updated1'])]['Genre_1'].unique()

array(['epic', 'adult hygiene exploitation', 'propaganda',
       'ww1 propaganda', 'swashbuckler', 'race', 'operetta', 'detective',
       'gangster', 'biblical', 'charlie chan', 'aviation', 'experimental',
       'educational', 'slapstick', 'espionage', 'horror, 3 d',
       'thriller, 3 d', 'sci_fi, 3 d', 'western, 3 d', 'science fiction',
       'p.o.w.', 'romance, trilogy', 'paramount. biblical epic',
       'disaster', 'drama, epic', 'adult', 'epic, biography', 'travel',
       'war, satire', 'comedy, satire', 'slasher', 'comedy, parody',
       'outlaw biker', 'biker', 'anthology', 'war, biker',
       'animated, adult', 'disaster , adventure',
       'science fiction, thriller', 'adult, horror',
       'drama, television miniseries', 'historical, erotic',
       'disaster , sci_fi', 'comedy, anthology', '16\xa0mm', 'parody',
       'horror, slasher', 'comedy, cult', 'cult', 'anthology, drama',
       'avant garde', 'imax', 'student', 'disaster, sci_fi',
       'historical epic,

In [33]:
new_df[(new_df['Genre_updated']!=new_df['Genre_updated1'])]['Genre_updated'].unique()

array(['action,western,crime,short', 'fantasy,short', 'drama,historical',
       'drama,fantasy', 'drama,biography', 'drama,documentary',
       'adventure,serial', 'comedy,short', 'action,adventure',
       'drama,western', 'drama,comedy', 'drama,crime', 'drama,war',
       'drama,romance', 'comedy,crime', 'comedy,western',
       'thriller,crime', 'comedy,adventure', 'comedy,musical',
       'comedy,mystery', 'romance,musical', 'drama,spy', 'crime,mystery',
       'horror,adventure', 'comedy,historical', 'comedy,romance',
       'sports,short', 'musical,biography', 'western,musical',
       'drama,sports', 'romance,western', 'drama,adventure',
       'musical,fantasy', 'comedy,thriller', 'war,spy', 'comedy,spy',
       'horror,sci_fi', 'thriller,spy', 'comedy,horror', 'comedy,sci_fi',
       'animated,short', 'drama,musical', 'western,serial',
       'comedy,action', 'crime,musical', 'thriller,fantasy',
       'comedy,animated', 'spy,serial', 'documentary,war', 'comedy,war',
       '